In [ ]:
# https://www.bfs.admin.ch/bfs/fr/home/services/recherche/api.html
# https://dam-api.bfs.admin.ch/hub/swagger-ui/index.html
# communes
# https://www.bfs.admin.ch/bfs/fr/home/services/recherche/api/api-gemeinde.assetdetail.15224055.html

# motorisation
# https://www.pxweb.bfs.admin.ch/sq/ccc96ddd-61a9-49ab-ad7f-dabbc934e8c0

import requests

# Population
dataset_url = "https://www.pxweb.bfs.admin.ch/api/v1/en/px-x-0102030000_101/px-x-0102030000_101.px"
dataset_url = "https://www.pxweb.bfs.admin.ch/api/v1/fr/px-x-1103020100_165/px-x-1103020100_165.px"

metadata_response = requests.get(dataset_url)
metadata_json = metadata_response.json()

response = requests.post(
    dataset_url,
    json={
        "query": [
            {
                "code": "Treibstoff",
                "selection": {
                    "filter": "item",
                    "values": ["100", "200", "300", "400", "500", "600", "9900"],
                },
            }
        ],
        "response": {"format": "json-stat"},
    },
)
json = response.json()
print(json)


response_canton = requests.post(
    dataset_url,
    json={
        "query": [
            {
                "code": "Kanton",
                "selection": {
                    "filter": "item",
                    "values": [
                        "1",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6",
                        "7",
                        "8",
                        "9",
                        "10",
                        "11",
                        "12",
                        "13",
                        "14",
                        "15",
                        "16",
                        "17",
                        "18",
                        "19",
                        "20",
                        "21",
                        "22",
                        "23",
                        "24",
                        "25",
                        "26",
                        "27",
                    ],
                },
            },
            {
                "code": "Treibstoff",
                "selection": {
                    "filter": "item",
                    "values": ["100", "200", "300", "400", "500", "600", "9900"],
                },
            },
        ],
        "response": {"format": "json-stat"},
    },
)
json_canton = response_canton.json()

In [ ]:
for elem in json["dataset"]["dimension"]["id"]:
    print("====")
    for k, v in json["dataset"]["dimension"][elem].items():
        print(f"{k}: {v}")

In [ ]:
import pandas as pd

values = {"values": json["dataset"]["value"]}
length = 1
total_length = len(json["dataset"]["value"])
for dimension_id in json["dataset"]["dimension"]["id"]:
    dimension = json["dataset"]["dimension"][dimension_id]

    current_length = len(dimension["category"]["index"])
    nb = int(total_length / (length * current_length))

    v = list(json["dataset"]["value"])
    for x in range(length):
        for y, value in enumerate(dimension["category"]["label"].values()):
            for z in range(nb):
                v[x * current_length * nb + y * nb + z] = value

    values[dimension["label"]] = v

    length *= current_length

df = pd.DataFrame(values)
df
# print(json['dataset']['value'])

In [ ]:
elem_key = json["dataset"]["dimension"]["id"][0]
elem_dataset = json["dataset"]["dimension"][elem_key]
year_key = json["dataset"]["dimension"]["id"][1]
year_dataset = json["dataset"]["dimension"][year_key]
year_length = len(year_dataset["category"]["index"])
values_bokeh = {"year": year_dataset["category"]["label"].values()}
col_bokeh = []
for key, index in elem_dataset["category"]["index"].items():
    start = index * year_length
    values_bokeh[elem_dataset["category"]["label"][key]] = json["dataset"]["value"][
        start : start + year_length
    ]
    col_bokeh.append(elem_dataset["category"]["label"][key])

df_bokeh = pd.DataFrame(values_bokeh)
print(col_bokeh)
print(df_bokeh)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="ticks")
sns.set_theme(style="darkgrid")

sns.set(rc={"figure.figsize": (4, 1)})
sns.lineplot(x="Année", y="values", hue="Carburant", data=df)

plt.yscale("log")
plt.ylabel("Carburant")
plt.xticks(rotation=25)

In [ ]:
import plotly.express as px

fig = px.line(df, x="Année", y="values", color="Carburant")
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
import bokeh.sampledata

bokeh.sampledata.download()

In [ ]:
pd.set_option("plotting.backend", "pandas_bokeh")

# fig = px.line(df, x="Année", y="values", color='Carburant')
df_bokeh.plot_bokeh.line(
    figsize=(800, 450),
    x="year",
    y=col_bokeh,
    title="Carburant",
    xlabel="Date",
    ylabel="Voitures",
    rangetool=True,
)
# y_axis_type='log'

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

fig = plt.figure()
ax = fig.gca(projection="3d")
X, Y, Z = axes3d.get_test_data(0.05)
ax.plot_surface(X, Y, Z, rstride=8, cstride=8, alpha=0.3)
cset = ax.contourf(X, Y, Z, zdir="z", offset=-100, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Z, zdir="x", offset=-40, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Z, zdir="y", offset=40, cmap=cm.coolwarm)

ax.set_xlabel("X")
ax.set_xlim(-40, 40)
ax.set_ylabel("Y")
ax.set_ylim(-40, 40)
ax.set_zlabel("Z")
ax.set_zlim(-100, 100)

plt.show()